# Pair Programming Limpieza II

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex.       
Si reducimos esto a una tabla para saber que tenemos ya limpito y que no.       
**Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

-  Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

In [53]:
import pandas as pd
import numpy as np
import re

In [54]:
df=pd.read_csv('files/attacks_limp1.csv', index_col=0)
df.head(2)

,year,country,age,species_,month,gender,dead
0,2018,usa,57,White shark,Jun,F,N
1,2018,brazil,18,Tiger shark,Jun,M,Y


    1  Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

In [ ]:
# imaginemos que el valor de una celda es el siguiente

'White shark, 3.5 m'

# tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
# Un patrón que podriamos usar es: 

patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

# de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies
#  que queremos "encontrar". 

In [55]:
def obtener_especie( col):
    patron1=r".*[Ww](hite|HITE).*"
    patron2=r".*[Tt](iger|IGER).*"
    patron3=r".*[Gg](rey|REY).*"
    patron4=r".*[Ll](emon|EMON).*"
    patron5=r".*[Bb](ull|ULL).*"
    try:
        if len(re.findall(patron1,col))>0:
            return 'white'
        elif len(re.findall(patron2,col))>0:
            return 'tiger'
        elif len(re.findall(patron3,col))>0:
            return 'grey'
        elif len(re.findall(patron4,col))>0:
            return 'lemon'
        elif len(re.findall(patron5,col))>0:
            return 'bull'
        else:
            return 'unspecified'
    except:
        return np.nan

In [56]:
df['specie']=df['species_'].apply(obtener_especie)
df.head()

,year,country,age,species_,month,gender,dead,specie
0,2018,usa,57,White shark,Jun,F,N,white
1,2018,brazil,18,Tiger shark,Jun,M,Y,tiger
2,2018,usa,15,"Bull shark, 6'",May,M,N,bull
3,2018,australia,32,Grey reef shark,May,M,N,grey
4,2018,england,21,Invalid incident,May,M,N,unspecified


    2  Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

- Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:       
a.Edad en formato string      
b.Edades separadas por &, or, to, >       
c.Edades con ?        
Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:      


    1  Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen.      
    De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

    2 Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos.
Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, [aqui](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html).

#TypeError: expected string or bytes-like object, solucion:
df['nombre_columna'] = df['nombre_columna'].astype(str)

In [85]:
def años (col):
    try:
        int(col)
        return int(col)       
    except:
        try:
            list= re.findall('\d+',col)
            if len(list)>1:
                suma=0
                for i in list:
                    suma+=int(i)
                    longitud=int(len(list))
                    return suma/longitud
            elif len(list)==1:
                return int(list[0])
            else:
                return np.nan
        except:
            return np.nan

In [86]:
df['edad']=df['age'].apply(años)
df.head()

,year,country,age,species_,month,gender,dead,specie,edad
0,2018,usa,57,White shark,Jun,F,N,white,57.0
1,2018,brazil,18,Tiger shark,Jun,M,Y,tiger,18.0
2,2018,usa,15,"Bull shark, 6'",May,M,N,bull,15.0
3,2018,australia,32,Grey reef shark,May,M,N,grey,32.0
4,2018,england,21,Invalid incident,May,M,N,unspecified,21.0


In [87]:
df['age'].value_counts().sum()

1516

In [88]:
df['age'].isnull().sum()

138

In [89]:
df['edad'].value_counts().sum()

1512

In [90]:
df['edad'].isnull().sum()


142

In [91]:
#Tenemos 4 valores nulos más en al columna nueva unificada de edades

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1654 entries, 0 to 1671
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      1654 non-null   int64  
 1   country   1644 non-null   object 
 2   age       1516 non-null   object 
 3   species_  1543 non-null   object 
 4   month     1605 non-null   object 
 5   gender    1639 non-null   object 
 6   dead      1555 non-null   object 
 7   specie    1543 non-null   object 
 8   edad      1512 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 129.2+ KB


    3 Por último cambiad el tipo de la columna de string a integer.

In [ ]:
df['edad']=df['edad'].astype('int64')

- Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [95]:
df.to_csv('files/attacks_limp2')